In [2]:
import pandas as pd
# 📥 1. Cargar el dataset completo
df = pd.read_csv("datasetwg.csv")

In [7]:
df

,customer_id,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,stock_final,cat1,cat2,...,"ROLLING_MEAN(periodo, plan_precios_cuidados)","ROLLING_MEAN(periodo, sku_size)","ROLLING_MEAN(periodo, stock_final)","ROLLING_MEAN(periodo, tn)",tn_lag_1,tn_lag_2,tn_lag_3,tn_lag_4,tn_lag_5,tn_delta
0,10001,20001,2017-01-01,0.0,11.0,99.43861,99.43861,0.00000,HC,ROPA LAVADO,...,0.0,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000
1,10001,20002,2017-01-01,0.0,17.0,90.13504,87.64856,0.00000,HC,ROPA LAVADO,...,0.0,3000.000000,0.000000,99.438610,99.43861,0.00000,0.00000,0.00000,0.0,-11.79005
2,10001,20003,2017-01-01,0.0,25.0,100.21284,100.21284,0.00000,FOODS,ADEREZOS,...,0.0,3000.000000,0.000000,93.543585,87.64856,99.43861,0.00000,0.00000,0.0,12.56428
3,10001,20004,2017-01-01,0.0,13.0,21.73954,21.73954,0.00000,FOODS,ADEREZOS,...,0.0,2158.333333,0.000000,95.766670,100.21284,87.64856,99.43861,0.00000,0.0,-78.47330
4,10001,20005,2017-01-01,0.0,0.0,0.00000,0.00000,0.00000,FOODS,ADEREZOS,...,0.0,1238.333333,0.000000,69.866980,21.73954,100.21284,87.64856,99.43861,0.0,-21.73954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9460975,10636,20282,2019-12-01,0.0,0.0,0.00000,0.00000,0.00000,PC,CABELLO,...,0.0,50.000000,2.205257,0.000000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000
9460976,10636,20386,2019-12-01,0.0,0.0,0.00000,0.00000,1.22801,PC,DEOS,...,0.0,36.666667,1.020657,0.000000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000
9460977,10636,20838,2019-12-01,0.0,0.0,0.00000,0.00000,0.57636,PC,DEOS,...,0.0,53.333333,0.913477,0.000000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000
9460978,10637,20657,2019-12-01,0.0,0.0,0.00000,0.00000,11.76647,HC,HOGAR,...,0.0,56.666667,0.601457,0.000000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000


In [4]:
print(df_train.isnull().sum().sort_values(ascending=False).head(20))


customer_id                                     0
ROLLING_MEAN(periodo, cust_request_tn)          0
LAG(cust_request_qty, periodo)                  0
LAG(cust_request_tn, periodo)                   0
LAG(plan_precios_cuidados, periodo)             0
LAG(sku_size, periodo)                          0
LAG(stock_final, periodo)                       0
LAG(tn, periodo)                                0
MONTH(periodo)                                  0
ROLLING_MEAN(periodo, cust_request_qty)         0
ROLLING_MEAN(periodo, plan_precios_cuidados)    0
LAG(cat2, periodo)                              0
ROLLING_MEAN(periodo, sku_size)                 0
ROLLING_MEAN(periodo, stock_final)              0
ROLLING_MEAN(periodo, tn)                       0
tn_lag_1                                        0
tn_lag_2                                        0
tn_lag_3                                        0
tn_lag_4                                        0
tn_lag_5                                        0


In [5]:
print(df_train['tn'].nunique())
print(df_train['tn'].value_counts(dropna=False).head())


91943
0.00000    7167499
0.01638      13930
0.00218      12908
0.00983      11889
0.01092      10714
Name: tn, dtype: int64


In [6]:
print("Cantidad de filas:", df_train.shape[0])


Cantidad de filas: 9460980


In [11]:
import pandas as pd
from autogluon.tabular import TabularPredictor

# 📥 1. Cargar el dataset
df = pd.read_csv("datasetwg.csv")
df['periodo'] = pd.to_datetime(df['periodo'])

# 📦 2. Separar entrenamiento hasta 2019-12
df_train = df[df['periodo'] <= '2019-12-01'].copy()
df_train = df_train[df_train['tn'] > 0].copy()  # eliminar nulos o ceros

# 📦 3. Crear df_pred solo con product_id únicos
df_pred = df[['product_id']].drop_duplicates().copy()
df_pred['periodo'] = pd.to_datetime('2020-02-01')

# ➕ 4. Agregar columnas faltantes del train al pred
cols_faltantes = [col for col in df_train.columns if col not in df_pred.columns and col != 'tn']
for col in cols_faltantes:
    df_pred[col] = None

# 🧮 5. Reordenar columnas para que coincidan
df_pred = df_pred[df_train.drop(columns='tn').columns]

# ⚙️ 6. Entrenar AutoGluon
predictor = TabularPredictor(label='tn', path='autogluon_pycaret_style').fit(
    train_data=df_train,
    time_limit=600,
    presets='best_quality'
)

# 🔮 7. Predecir sobre el set simulado para febrero
df_pred['tn'] = predictor.predict(df_pred)

# 💾 8. Guardar resultado
resultado = df_pred[['product_id', 'tn']].rename(columns={'tn': 'tn_predicho'})
resultado.to_csv("predicciones_febrero2020_autogluon.csv", index=False)

print("✅ CSV generado: predicciones_febrero2020_autogluon.csv")


Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 600s
AutoGluon will save models to "autogluon_pycaret_style\"
AutoGluon Version:  0.8.2
Python Version:     3.9.23
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
Disk Space Avail:   1550.56 GB / 1999.59 GB (77.5%)
Train Data Rows:    2293481
Train Data Columns: 42
Label Column: tn
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (547.87849, 0.0001, 0.48948, 3.39599)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoM

✅ CSV generado: predicciones_febrero2020_autogluon.csv


In [12]:
import pandas as pd
from autogluon.tabular import TabularPredictor

# 📥 1. Cargar el dataset
df = pd.read_csv("datasetwg.csv")
df['periodo'] = pd.to_datetime(df['periodo'])

# 🧹 2. Separar datos de entrenamiento (hasta dic-2019)
df_train = df[df['periodo'] <= '2019-12-01'].copy()

# 🧠 3. Para predicción: obtener últimas features por product_id
columnas_clave = ['product_id']
columnas_static = [col for col in df.columns if col != 'tn' and col != 'periodo']
df_pred = (
    df[df['periodo'] == '2019-12-01']
    .drop_duplicates(subset='product_id')
    .copy()
)

# Ajustar columna periodo a febrero 2020
df_pred['periodo'] = pd.to_datetime('2020-02-01')

# 💡 Asegurarse de que no hay target en df_pred
df_pred = df_pred[[col for col in df_pred.columns if col != 'tn']]

# 🧼 4. Filtrar entrenamiento para registros útiles
df_train = df_train[df_train['tn'] > 0].copy()

# 🏁 5. Entrenar AutoGluon
predictor = TabularPredictor(label='tn', path='autogluon_model_final').fit(
    train_data=df_train,
    time_limit=600,
    presets='best_quality'
)

# 🔮 6. Predecir
df_pred['tn'] = predictor.predict(df_pred)

# 📊 7. Guardar resultados
df_resultado = df_pred[['product_id', 'tn']].rename(columns={'tn': 'tn_predicho'})
df_resultado.to_csv("predicciones_febrero2020_autogluon1.csv", index=False)
print("✅ CSV generado: predicciones_febrero2020_autogluon1.csv")


Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 600s
AutoGluon will save models to "autogluon_model_final\"
AutoGluon Version:  0.8.2
Python Version:     3.9.23
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
Disk Space Avail:   1549.77 GB / 1999.59 GB (77.5%)
Train Data Rows:    2293481
Train Data Columns: 42
Label Column: tn
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (547.87849, 0.0001, 0.48948, 3.39599)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLP

[1000]	valid_set's rmse: 1.73066
[1000]	valid_set's rmse: 1.42408


	Ran out of time, early stopping on iteration 1032. Best iteration is:
	[1009]	valid_set's rmse: 1.42386


[1000]	valid_set's rmse: 1.4911


	Ran out of time, early stopping on iteration 1127. Best iteration is:
	[1104]	valid_set's rmse: 1.48294


[1000]	valid_set's rmse: 1.41251


	Ran out of time, early stopping on iteration 1248. Best iteration is:
	[1248]	valid_set's rmse: 1.40193
	-1.4612	 = Validation score   (-root_mean_squared_error)
	344.96s	 = Training   runtime
	18.6s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ... Training model for up to 5.9s of the 200.72s of remaining time.
	Memory not enough to fit LGBModel folds in parallel. Will do sequential fitting instead. 	Consider decreasing folds trained in parallel by passing num_folds_parallel to ag_args_ensemble when calling predictor.fit
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Ran out of time, early stopping on iteration 1. Best iteration is:
	[1]	valid_set's rmse: 3.31793
	Time limit exceeded... Skipping LightGBM_BAG_L1.
Completed 1/20 k-fold bagging repeats ...
Fitting model: WeightedEnsemble_L2 ... Training model for up to 360.0s of the 191.87s of remaining time.
	-1.4612	 = Validation score   (-root_mean_squared_error)
	0.06s	 = Training   r

✅ CSV generado: predicciones_febrero2020_autogluon1.csv
